# Complex Scenario Analysis

This notebook demonstrates how to access and use the functionalities of **Scenario Analysis** which are part of our **Yiedbook** module within LSEG Financial Analytics SDK. The notebook outlines the broadest complexity when performing scenario analysis, where all the possible set of arguments are present.

The main goal of this notebook is to explore and display an approach in fetching Scenario Analysis information in its most complex form by using any of 4 available methods:

1. Synchronous POST
2. Synchronous GET
3. Asynchronous POST
4. Asynchronous GET

## Imports

Import the following necessary modules:

- *request_scenario_calculation_sync* - main Synchronous POST method
- *ScenarioCalcGlobalSettings* - Class wrapper for Global Settings 
- *ScenarioCalcInput* - Class wrapper for Input parameters
- *Scenario* - Class wrapper for defining various Scenarios
- *json* - for display purposes  

In [1]:
from lseg_analytics.yield_book_rest import (
    request_scenario_calculation_sync,
    request_get_scen_calc_sys_scen_sync,
    request_scenario_calculation_async,
    request_get_scen_calc_sys_scen_async,
    get_result,
    Volatility,
    StructureNote,
    CurveTypeAndCurrency,
    LossSettings,
    Vector,
    MonthRatePair,
    RestPrepaySettings,
    ScenarioCalcGlobalSettings,
    Scenario,
    ScenarioDefinition,
    SystemScenario,
    UserScenario,
    ApimCurveShift,
    CurveMultiShift,
    ScenAbsoluteCurvePoint,
    ScenarioVolatility,
    SwaptionVolatility,
    SwaptionVolItem,
    CapVolatility,
    CapVolItem,
    ScenarioCalcInput,
    SettlementInfo,
    PricingScenario,
    CustomScenario,
    CmbsPrepayment,
    Balloon,
    HorizonInfo,
    ScenarioCalcFloaterSettings,
    HecmSettings,
    ScenCalcExtraSettings,
    ScenPartials,
)
import json as js
import time

## Data Preparation

Cash Flow Post methods consume three input arguments: 
- Global settings 
    - settings applicable in overall Scenario Analysis
- Inputs
    - separate parameters to describe and enable specific SA information
- Scenario
    - individual definition of Scenarios used for analysis

In [2]:
# request_scenario_calculation_async
global_settings = ScenarioCalcGlobalSettings(
            pricing_date="2025-01-01",
            use_previous_close=False,
            use_live_data=False,
            calc_horizon_effective_measures=False,
            calc_horizon_option_measures=True,
            calc_scenario_cash_flow=True,
            calc_prepay_sensitivity=True,
            current_coupon_rates="MOATS",
            horizon_months=1,
            horizon_days=5,
            use_stochastic_hpa=True,
            use_five_point_convexity=True,
            use_core_logic_group_model=True,
            use_muni_non_call_curve=True,
            use_muni_tax_settings=True,
            use_ois=False,
            core_logic_collateral="DEFAULT",
        )
scenario = Scenario(
            scenario_id="ScenID1",
            scenario_title="ScenTest1",
            timing="AtHorizon",
            reinvestment_rate="1.3",
            definition=ScenarioDefinition(
                system_scenario=SystemScenario(name="BEARFLAT100"),
                user_scenario=UserScenario(
                    shift_type="Forward",
                    interpolation_type="PrincipalComponents",
                    swap_spread_const=True,
                    curve_shifts=[ApimCurveShift(year=1, value=1.1)],
                    curve_multi_shifts=[CurveMultiShift(curve_shifts=[ApimCurveShift(year=1, value=1.1)])],
                    curve_points=[
                        ScenAbsoluteCurvePoint(
                            term=1, rate=1.1, spot_rate=1.231, forward_rate=1.105, discount_factor=0.8
                        )
                    ],
                ),
            ),
            volatility=ScenarioVolatility(
                term_unit="MONTH",
                value_type="ABS",
                parallel_shift=1.1,
                swaption_volatility=SwaptionVolatility(
                    value_type="ABS", values_property=[SwaptionVolItem(expiration=1.121, value=1.011, term=3.3)]
                ),
                cap_volatility=CapVolatility(
                    value_type="ABS", values_property=[CapVolItem(expiration=1.121, value=1.011)]
                ),
            ),
            current_coupon_spread_change=1.1,
        )
input = ScenarioCalcInput(
    identifier="US742718AV11",
    id_type="ISIN",
    curve=CurveTypeAndCurrency(
        curve_type="GVT",
        currency="USD",
        retrieve_curve=True,
        snapshot="4PM",
    ),
    volatility=Volatility(
        type="Single",
        rate="1.1",
        structure_note=StructureNote(
            pricing="ASSETSWAP", callable_zero_pricing="NETPROCEED"
        ),
    ),
    settlement_info=SettlementInfo(
        level="100",
        settlement_type="CUSTOM",
        settlement_date="2025-01-01",
        prepay=RestPrepaySettings(
            type="ABS",
            rate=1.1,
            vector=Vector(
                interpolation_type="INTERPOLATED",
                index="PRIM",
                values_property=[MonthRatePair(month=1, rate=1.1)],
            ),
            model_to_balloon=True,
        ),
        loss_settings=LossSettings(
            default_type="CDR",
            default_rate=1.1,
            default_vector=Vector(
                interpolation_type="INTERPOLATED",
                index="PRIM",
                values_property=[MonthRatePair(month=1, rate=1.1)],
            ),
            severity_type="MODEL",
            severity_rate=1.1,
            severity_vector=Vector(
                interpolation_type="INTERPOLATED",
                index="PRIM",
                values_property=[MonthRatePair(month=1, rate=1.1)],
            ),
            recovery_lag=1,
            delinquency_type="MODEL",
            delinquency_rate=1.1,
            delinquency_vector=Vector(
                interpolation_type="INTERPOLATED",
                index="PRIM",
                values_property=[MonthRatePair(month=1, rate=1.1)],
            ),
            use_model_loan_modifications=True,
            ignore_insurance=True,
        ),
        cmbs_scenario=PricingScenario(
            primary=True,
            type="CPJ",
            rate=1.1,
            system_scenario_name="name",
            custom_scenario=CustomScenario(
                assume_call=True,
                delay=True,
                delay_balloon_maturity=True,
                defeasance="AUTO",
                prepayment=CmbsPrepayment(
                    rate_during_yield_to_maturity=1.017,
                    rate_after_yield_to_maturity=0.987,
                    rate_during_premium=2.331,
                ),
                defaults=Balloon(
                    percent=0.8,
                    loss_severity=0.8,
                    recovery_period=1,
                    loss_type="CDR",
                    loss_rate=0.8,
                    month_to_extend=2,
                    loss_vector=Vector(
                        interpolation_type="INTERPOLATED",
                        index="PRIM",
                        values_property=[MonthRatePair(month=1, rate=1.1)],
                    ),
                ),
                balloon_extend=Balloon(
                    percent=0.8,
                    loss_severity=0.8,
                    recovery_period=1,
                    loss_type="CDR",
                    loss_rate=0.8,
                    month_to_extend=2,
                    loss_vector=Vector(
                        interpolation_type="INTERPOLATED",
                        index="PRIM",
                        values_property=[MonthRatePair(month=1, rate=1.1)],
                    ),
                ),
                balloon_default=Balloon(
                    percent=0.8,
                    loss_severity=0.8,
                    recovery_period=1,
                    loss_type="CDR",
                    loss_rate=0.8,
                    month_to_extend=2,
                    loss_vector=Vector(
                        interpolation_type="INTERPOLATED",
                        index="PRIM",
                        values_property=[MonthRatePair(month=1, rate=1.1)],
                    ),
                ),
            ),
        ),
    ),
    horizon_info=[
        HorizonInfo(
            scenario_id="ScenID1",
            level="100",
            prepay=RestPrepaySettings(
                type="ABS",
                rate=1.1,
                vector=Vector(
                    interpolation_type="INTERPOLATED",
                    index="PRIM",
                    values_property=[MonthRatePair(month=1, rate=1.1)],
                ),
                model_to_balloon=True,
            ),
            loss_settings=LossSettings(
                default_type="CDR",
                default_rate=1.1,
                default_vector=Vector(
                    interpolation_type="INTERPOLATED",
                    index="PRIM",
                    values_property=[MonthRatePair(month=1, rate=1.1)],
                ),
                severity_type="MODEL",
                severity_rate=1.1,
                severity_vector=Vector(
                    interpolation_type="INTERPOLATED",
                    index="PRIM",
                    values_property=[MonthRatePair(month=1, rate=1.1)],
                ),
                recovery_lag=1,
                delinquency_type="MODEL",
                delinquency_rate=1.1,
                delinquency_vector=Vector(
                    interpolation_type="INTERPOLATED",
                    index="PRIM",
                    values_property=[MonthRatePair(month=1, rate=1.1)],
                ),
                use_model_loan_modifications=True,
                ignore_insurance=True,
            ),
            cmbs_scenario=PricingScenario(
                primary=True,
                type="CPJ",
                rate=1.1,
                system_scenario_name="name",
                custom_scenario=CustomScenario(
                    assume_call=True,
                    delay=True,
                    delay_balloon_maturity=True,
                    defeasance="AUTO",
                    prepayment=CmbsPrepayment(
                        rate_during_yield_to_maturity=1.017,
                        rate_after_yield_to_maturity=0.987,
                        rate_during_premium=2.331,
                    ),
                    defaults=Balloon(
                        percent=0.8,
                        loss_severity=0.8,
                        recovery_period=1,
                        loss_type="CDR",
                        loss_rate=0.8,
                        month_to_extend=2,
                        loss_vector=Vector(
                            interpolation_type="INTERPOLATED",
                            index="PRIM",
                            values_property=[MonthRatePair(month=1, rate=1.1)],
                        ),
                    ),
                    balloon_extend=Balloon(
                        percent=0.8,
                        loss_severity=0.8,
                        recovery_period=1,
                        loss_type="CDR",
                        loss_rate=0.8,
                        month_to_extend=2,
                        loss_vector=Vector(
                            interpolation_type="INTERPOLATED",
                            index="PRIM",
                            values_property=[MonthRatePair(month=1, rate=1.1)],
                        ),
                    ),
                    balloon_default=Balloon(
                        percent=0.8,
                        loss_severity=0.8,
                        recovery_period=1,
                        loss_type="CDR",
                        loss_rate=0.8,
                        month_to_extend=2,
                        loss_vector=Vector(
                            interpolation_type="INTERPOLATED",
                            index="PRIM",
                            values_property=[MonthRatePair(month=1, rate=1.1)],
                        ),
                    ),
                ),
            ),
        )
    ],
    assume_call=True,
    horizon_py_method="Annualized ROR",
    floater_settings=ScenarioCalcFloaterSettings(
        use_forward_index=True, use_immediate_forward_shift=True
    ),
    hecm_settings=HecmSettings(
        draw_type="CONSTANT",
        draw_rate=1.1,
        draw_vector=Vector(
            interpolation_type="INTERPOLATED",
            index="PRIM",
            values_property=[MonthRatePair(month=1, rate=1.1)],
        ),
    ),
    extra_settings=ScenCalcExtraSettings(
        include_partials=True,
        partials=ScenPartials(
            curve_type="FORWARD",
            curve_shift=1.1,
            shock_type="SQUARE",
            partial_duration_years=[1.1, 1.2],
        ),
    ),
)

## Synchronous POST Data retrieval

Once the input arguments are ready, they can be passed through the main method for data retrieval

In [3]:
# Execute Post sync request with prepared inputs
sa_sync_post_response = request_scenario_calculation_sync(
                            global_settings=global_settings,
                            scenarios=[scenario],
                            input=[input],
                        )

## Synchronous GET Data retrieval

As opposed to the POST method, the GET method has the following mandatory arguments:
- identifier
- horizon py method
- curve type
- pricing date
- horizon level 
- existing scenario name. 

The remaining arguments are optional. 

In [4]:
# Formulate and execute the get request
sa_sync_get_response = request_get_scen_calc_sys_scen_sync(
            id='US742718AV11',
            h_py_method="OAS",
            curve_type="GVT",
            pricing_date="2025-01-01",
            level="100",
            scenario="/sys/scenario/Par/50"
        )

## Asynchronous POST Data retrieval

The main difference between the Sync and Async approach is that Async methods allow execution of code on the user's side while waiting for a response. Once execution is complete, the user can 'collect' the results whenever they need them.

So this approach has 2 steps:
1. Provide input data to execution method
2. Use the request_id received in step 1 to retrieve the result

In [5]:
# Request bond CF with async post
sa_async_post_response = request_scenario_calculation_async(
                            global_settings=global_settings,
                            scenarios=[scenario],
                            input=[input],
                        )

attempt = 1

while attempt < 10:

    from lseg_analytics.exceptions import ResourceNotFound
    try:
        # Request bond indic with async post
        async_post_results_response = get_result(request_id_parameter=sa_async_post_response.request_id)
        break
    except Exception as err:
        print(f"Attempt " + str(
            attempt) + " resulted in error retrieving results from:" + sa_async_post_response.request_id)
        if (isinstance(err, ResourceNotFound)
                and f"The result is not ready yet for requestID:{sa_async_post_response.request_id}" in str(err)):
            time.sleep(3)
            attempt += 1
        else:
            raise err

[ERROR]	[2025-09-18 08:58:04,429]	[MainThread]	[lseg_analytics.yield_book_rest]	[_functions.py:8525]	 Error get_result.
[ERROR]	[2025-09-18 08:58:04,430]	[MainThread]	[lseg_analytics]	[exceptions.py:78]	 Exception: Resource not found: code=404 {'meta': {'requestId': 'R-60863', 'status': 'RUNNING', 'responseType': 'SCENARIO_CALC', 'message': 'The result is not ready yet for requestID:R-60863', 'interval': 0.1}}
Attempt 1 resulted in error retrieving results from:R-60863


## Asynchronous GET Data retrieval

Difference between Sync and Async can be found in description above.

As Async execution is in queue-form it's not necessarily instantly finished. It can take time between submission of request and execution, and results are not in real-time.

Hence we present also the form of iterative results-request repetition to fetch the results as soon as they are ready.

**NOTE:** Alternative to this is wrapping the request in the job, and then using a method to check the job status. See the fundamentals notebook for information on jobs.

In [6]:
# Formulate and execute the get request by using instrument ID, Par_amount and job in which the calculation will be done
sa_async_get_response = request_get_scen_calc_sys_scen_async(
            id='US742718AV11',
            scenario="/sys/scenario/Par/50",
            pricing_date="2025-01-01",
            curve_type="GVT",
            h_py_method="OAS",
            h_level="100",
            level="100"
        )

# Due to async nature, code Will perform the fetch 10 times, as result is not always ready instantly, with 3 second lapse between attempts
attempt = 1

while attempt < 10:

    from lseg_analytics.exceptions import ResourceNotFound
    try:
        # Request bond indic with async post
        async_get_results_response = get_result(request_id_parameter=sa_async_get_response.request_id)
        break
    except Exception as err:
        print(f"Attempt " + str(
            attempt) + " resulted in error retrieving results from:" + sa_async_get_response.request_id)
        if (isinstance(err, ResourceNotFound)
                and f"The result is not ready yet for requestID:{sa_async_get_response.request_id}" in str(err)):
            time.sleep(3)
            attempt += 1
        else:
            raise err

[ERROR]	[2025-09-18 08:58:07,793]	[MainThread]	[lseg_analytics.yield_book_rest]	[_functions.py:8525]	 Error get_result.
[ERROR]	[2025-09-18 08:58:07,793]	[MainThread]	[lseg_analytics]	[exceptions.py:78]	 Exception: Resource not found: code=404 {'meta': {'requestId': 'R-60864', 'status': 'RUNNING', 'responseType': 'SCENARIO_CALC', 'message': 'The result is not ready yet for requestID:R-60864', 'interval': 0.1}}
Attempt 1 resulted in error retrieving results from:R-60864


## Display results

Results data is structured in JSON format so it can be easily displayed using print function.

### Synchronous POST results

In [7]:
# Print output in JSON format
print(js.dumps(obj=sa_sync_post_response, indent=4))

{
    "meta": {
        "status": "DONE",
        "requestId": "R-60861",
        "timeStamp": "2025-09-18T04:58:03Z",
        "responseType": "SCENARIO_CALC",
        "resultsStatus": "ALL"
    },
    "results": [
        {
            "isin": "US742718AV11",
            "cusip": "742718AV1",
            "ticker": "PG",
            "scenario": {
                "horizon": [
                    {
                        "oas": 174.6789,
                        "wal": 4.7222,
                        "vega": 0.0,
                        "price": 107.543878,
                        "yield": 6.1321,
                        "balance": 1.0,
                        "pylevel": "100.000000",
                        "duration": 3.856,
                        "fullPrice": 109.7661,
                        "returnCode": 0,
                        "scenarioID": "ScenID1",
                        "volatility": 2.2,
                        "actualPrice": 107.544,
                        "grossSpread"

### Synchronous GET results

In [8]:
# Print output in JSON format
print(js.dumps(obj=sa_sync_get_response, indent=4))

{
    "data": {
        "isin": "US742718AV11",
        "cusip": "742718AV1",
        "ticker": "PG",
        "scenario": {
            "horizon": [
                {
                    "oas": 361.951,
                    "wal": 4.8139,
                    "price": 98.053324,
                    "yield": 8.4961,
                    "balance": 1.0,
                    "duration": 3.8584,
                    "fullPrice": 99.542213,
                    "returnCode": 0,
                    "scenarioID": "/sys/scenario/Par/50",
                    "spreadDV01": 0.0,
                    "volatility": 16.0,
                    "actualPrice": 98.053,
                    "grossSpread": 361.3423,
                    "horizonDays": 0,
                    "marketValue": 99.542213,
                    "optionValue": 0.0,
                    "totalReturn": -1.91811763,
                    "dollarReturn": -1.94667627,
                    "convexityCost": 0.0,
                    "nominalSpread": 361

### Asynchronous POST results

In [9]:
# Print output in JSON format
print(js.dumps(obj=async_post_results_response, indent=4))

{
    "meta": {
        "status": "DONE",
        "requestId": "R-60863",
        "timeStamp": "2025-09-18T04:58:04Z",
        "responseType": "SCENARIO_CALC",
        "resultsStatus": "ALL"
    },
    "results": [
        {
            "isin": "US742718AV11",
            "cusip": "742718AV1",
            "ticker": "PG",
            "scenario": {
                "horizon": [
                    {
                        "oas": 174.6789,
                        "wal": 4.7222,
                        "vega": 0.0,
                        "price": 107.543878,
                        "yield": 6.1321,
                        "balance": 1.0,
                        "pylevel": "100.000000",
                        "duration": 3.856,
                        "fullPrice": 109.7661,
                        "returnCode": 0,
                        "scenarioID": "ScenID1",
                        "volatility": 2.2,
                        "actualPrice": 107.544,
                        "grossSpread"

### Asynchronous GET results

In [10]:
# Print output in JSON format
print(js.dumps(obj=async_get_results_response, indent=4))

{
    "data": {
        "isin": "US742718AV11",
        "cusip": "742718AV1",
        "ticker": "PG",
        "scenario": {
            "horizon": [
                {
                    "oas": 100.0,
                    "wal": 4.8139,
                    "price": 108.774734,
                    "yield": 5.8775,
                    "balance": 1.0,
                    "pylevel": "100.000000",
                    "duration": 3.9542,
                    "fullPrice": 110.263623,
                    "returnCode": 0,
                    "scenarioID": "/sys/scenario/Par/50",
                    "spreadDV01": 0.0,
                    "volatility": 16.0,
                    "actualPrice": 108.775,
                    "grossSpread": 99.4751,
                    "horizonDays": 0,
                    "marketValue": 110.263623,
                    "optionValue": 0.0,
                    "totalReturn": 8.64600454,
                    "dollarReturn": 8.77473393,
                    "convexityCost": 0